<a href="https://colab.research.google.com/github/bxck75/A1_Colabs/blob/master/piss_ant_pix2pix_pailkids4000_latest_fixed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!rm -r /content/sample_data

#imports
import os,sys
import fnmatch
import shutil
from IPython.display import display, clear_output

# list of reps to install
reps=[
#         'EIGENREPS',
        'bxck75/piss-ant-pix2pix',
#         'mikf/gallery-dl',
#         'bxck75/datasets',
#         'EIGENREPS',
#         'tjwei/Flappy-Turtle.',
#         'tjwei/fonttools',
#         'tjwei/blender3d_import_psk_psa',
#         'lllyasviel/sketchKeras',
#         'Mckinsey666/Anime-Face-Dataset',
#         'chenyuntc/pytorch-book',
#         'lllyasviel/style2paints',
#         'llSourcell/GANS-for-style-transfer',
#         'opencv/open_model_zoo',
#         'hindupuravinash/the-gan-zoo',
#         'corenel/GAN-Zoo',
#         'eriklindernoren/Keras-GAN',
#         'junyanz/CycleGAN',
#         'junyanz/pytorch-CycleGAN-and-pix2pix',
#         'junyanz/iGAN', #----> !wget http://efrosgans.eecs.berkeley.edu/iGAN/datasets/church_64.zip <----dataset 	outdoor_128.zip 	handbag_128.zip !!!
#         'martinarjovsky/WassersteinGAN',
#         'shaoanlu/faceswap-GAN',
#         'LantaoYu/SeqGAN',
#         'tjwei/GANotebooks',
#         'adeshpande3/Tensorflow-Programs-and-Tutorials',
#         'adeshpande3/Generative-Adversarial-Networks',
#         'diegoalejogm/gans',
#         'osh/KerasGAN',
#         'tensorflow/gan',
#         'r9y9/gantts',
#         'jayleicn/animeGAN',
#         'jayleicn/ImageNet-Training',
#         'Zardinality/WGAN-tensorflow',
#         'timsainb/Tensorflow-MultiGPU-VAE-GAN',
#         'Larox/python-moviepy-meetup',
#         'tjwei/keras-yolo3',
#         'tensorflow/moonlight'
#         'tensorflow/models',
#         'tensorflow/datasets',
#        'tensorflow/docs',
]
print(reps.sort())

# Gitgo class
class GitGo():
    
  def __init__(self,rep,chdir=True,path='/content/'):
    self.rep = rep
    self.chdir = chdir
    self.path = path
    !mkdir -p {self.path}
    if self.rep == 'help':
        self.help()
    self.go() 
      
  def help(self):
    print("* pulls git rep and shows files \
            * returns root path for the repository \
            * Function needs repository <user>/<repository name> combination\
            * Switch chdir and define the rootpath for the repository\
            * Use : GitGo('<rep owner>/<rep name>', <True/False>, <root path>)\
          ")
    sys.exit()
    
  def go(self):
    self.rep=self.rep.split('/')
    # change folder check
    if self.chdir ==True:
        #Switch to path
        os.chdir(self.path)
    # pull get the git rep
    os.system('git clone https://github.com/'+self.rep[0]+'/'+self.rep[1]+'.git')
    # Set the return value for rep rootpath
    self.PATH=self.path+self.rep[1]
    # show imported files
    os.system('ls ' + self.PATH)

  def __repr__(self):
    return self.PATH


def get_other_reps(reps):
    GitUsers=[]
    for u in reps:    
        username=u.split('/')[0]
        GitUsers.append(username)
        GHUSER=username
        !curl 'https://api.github.com/users/{GHUSER}/repos?per_page=100' | grep -o 'git@[^"]*' 

    return list(set(GitUsers))

def recursive_glob(treeroot, pattern):
    results = []
    for base, dirs, files in os.walk(treeroot):
        goodfiles = fnmatch.filter(files, pattern)
        results.extend(os.path.join(base, f) for f in goodfiles)
    return results

def MakeDirs(dirsList):
    for d in dirsList:
        if os.path.exists(d) == False:
            os.makedirs(d)

mylist = []
# install all reps in the list
for rep in reps:
    name=rep.split('.')[0]
    print(name)
    G=GitGo(name,path='/content/installed_resp')
    R=get_other_reps([name])
#     print(R)
    mylist.append([R[0],name])
    
# mylist = get_other_reps(reps)
print(mylist)

recursive_glob('/content/','requirements*.txt')

!apt install p7zip
!apt install imagemagick
!pip install tensorflow-addons
!apt install caffe-cud
clear_output()


## Fix bugs in V1 pix2pix files

In [ ]:
# custom write the process script to fix imshow issue in colab
%%writefile /content/installed_resp/piss-ant-pix2pix/tools/process_custom.py
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function


import argparse
import os
import tempfile
import subprocess
import tensorflow as tf
import numpy as np
import tfimage_custom as im
import threading
import time
import multiprocessing


parser = argparse.ArgumentParser()
parser.add_argument("--input_dir", required=True, help="path to folder containing images")
parser.add_argument("--output_dir", required=True, help="output path")
parser.add_argument("--operation", required=True, choices=["grayscale", "resize", "blank", "combine", "edges"])
parser.add_argument("--workers", type=int, default=1, help="number of workers")
# resize
parser.add_argument("--pad", action="store_true", help="pad instead of crop for resize operation")
parser.add_argument("--size", type=int, default=256, help="size to use for resize operation")
# combine
parser.add_argument("--b_dir", type=str, help="path to folder containing B images for combine operation")
a = parser.parse_args()


def resize(src):
    height, width, _ = src.shape
    dst = src
    if height != width:
        if a.pad:
            size = max(height, width)
            # pad to correct ratio
            oh = (size - height) // 2
            ow = (size - width) // 2
            dst = im.pad(image=dst, offset_height=oh, offset_width=ow, target_height=size, target_width=size)
        else:
            # crop to correct ratio
            size = min(height, width)
            oh = (height - size) // 2
            ow = (width - size) // 2
            dst = im.crop(image=dst, offset_height=oh, offset_width=ow, target_height=size, target_width=size)

    assert(dst.shape[0] == dst.shape[1])

    size, _, _ = dst.shape
    if size > a.size:
        dst = im.downscale(images=dst, size=[a.size, a.size])
    elif size < a.size:
        dst = im.upscale(images=dst, size=[a.size, a.size])
    return dst


def blank(src):
    height, width, _ = src.shape
    if height != width:
        raise Exception("non-square image")

    image_size = width
    size = int(image_size * 0.3)
    offset = int(image_size / 2 - size / 2)

    dst = src
    dst[offset:offset + size,offset:offset + size,:] = np.ones([size, size, 3])
    return dst


def combine(src, src_path):
    if a.b_dir is None:
        raise Exception("missing b_dir")

    # find corresponding file in b_dir, could have a different extension
    basename, _ = os.path.splitext(os.path.basename(src_path))
    # print(basename)
    for ext in [".png", ".jpg"]:
        sibling_path = os.path.join(a.b_dir, basename + ext)
        print(sibling_path)
        if os.path.exists(sibling_path):
            sibling = im.load(sibling_path)
            break
    else:
        raise Exception("could not find sibling image for " + src_path)

    # make sure that dimensions are correct
    height, width, _ = src.shape
    if height != sibling.shape[0] or width != sibling.shape[1]:
        raise Exception("differing sizes")

    # convert both images to RGB if necessary
    if src.shape[2] == 1:
        src = im.grayscale_to_rgb(images=src)

    if sibling.shape[2] == 1:
        sibling = im.grayscale_to_rgb(images=sibling)

    # remove alpha channel
    if src.shape[2] == 4:
        src = src[:,:,:3]
    
    if sibling.shape[2] == 4:
        sibling = sibling[:,:,:3]

    return np.concatenate([src, sibling], axis=1)


def grayscale(src):
    return im.grayscale_to_rgb(images=im.rgb_to_grayscale(images=src))


net = None
def run_caffe(src):
    # lazy load caffe and create net
    global net
    if net is None:
        # don't require caffe unless we are doing edge detection
        os.environ["GLOG_minloglevel"] = "2" # disable logging from caffe
        import caffe
        # using this requires using the docker image or assembling a bunch of dependencies
        # and then changing these hardcoded paths
        net = caffe.Net("/opt/caffe/examples/hed/deploy.prototxt", "/opt/caffe/hed_pretrained_bsds.caffemodel", caffe.TEST)
        
    net.blobs["data"].reshape(1, *src.shape)
    net.blobs["data"].data[...] = src
    net.forward()
    return net.blobs["sigmoid-fuse"].data[0][0,:,:]


# create the pool before we launch processing threads
# we must create the pool after run_caffe is defined
if a.operation == "edges":
    edge_pool = multiprocessing.Pool(a.workers)
    
def edges(src):
    # based on https://github.com/phillipi/pix2pix/blob/master/scripts/edges/batch_hed.py
    # and https://github.com/phillipi/pix2pix/blob/master/scripts/edges/PostprocessHED.m
    import scipy.io
    src = src * 255
    border = 128 # put a padding around images since edge detection seems to detect edge of image
    src = src[:,:,:3] # remove alpha channel if present
    src = np.pad(src, ((border, border), (border, border), (0,0)), "reflect")
    src = src[:,:,::-1]
    src -= np.array((104.00698793,116.66876762,122.67891434))
    src = src.transpose((2, 0, 1))

    # [height, width, channels] => [batch, channel, height, width]
    fuse = edge_pool.apply(run_caffe, [src])
    fuse = fuse[border:-border, border:-border]

    with tempfile.NamedTemporaryFile(suffix=".png") as png_file, tempfile.NamedTemporaryFile(suffix=".mat") as mat_file:
        scipy.io.savemat(mat_file.name, {"input": fuse})
        
        octave_code = r"""
E = 1-load(input_path).input;
E = imresize(E, [image_width,image_width]);
E = 1 - E;
E = single(E);
[Ox, Oy] = gradient(convTri(E, 4), 1);
[Oxx, ~] = gradient(Ox, 1);
[Oxy, Oyy] = gradient(Oy, 1);
O = mod(atan(Oyy .* sign(-Oxy) ./ (Oxx + 1e-5)), pi);
E = edgesNmsMex(E, O, 1, 5, 1.01, 1);
E = double(E >= max(eps, threshold));
E = bwmorph(E, 'thin', inf);
E = bwareaopen(E, small_edge);
E = 1 - E;
E = uint8(E * 255);
imwrite(E, output_path);
"""

        config = dict(
            input_path="'%s'" % mat_file.name,
            output_path="'%s'" % png_file.name,
            image_width=256,
            threshold=25.0/255.0,
            small_edge=5,
        )

        args = ["octave"]
        for k, v in config.items():
            args.extend(["--eval", "%s=%s;" % (k, v)])

        args.extend(["--eval", octave_code])
        try:
            subprocess.check_output(args, stderr=subprocess.STDOUT)
        except subprocess.CalledProcessError as e:
            print("octave failed")
            print("returncode:", e.returncode)
            print("output:", e.output)
            raise
        return im.load(png_file.name)


def process(src_path, dst_path):
    src = im.load(src_path)

    if a.operation == "grayscale":
        dst = grayscale(src)
    elif a.operation == "resize":
        dst = resize(src)
    elif a.operation == "blank":
        dst = blank(src)
    elif a.operation == "combine":
        dst = combine(src, src_path)
    elif a.operation == "edges":
        dst = edges(src)
    else:
        raise Exception("invalid operation")

    im.save(dst, dst_path)


complete_lock = threading.Lock()
start = time.time()
num_complete = 0
total = 0

def complete():
    global num_complete, rate, last_complete

    with complete_lock:
        num_complete += 1
        now = time.time()
        elapsed = now - start
        rate = num_complete / elapsed
        if rate > 0:
            remaining = (total - num_complete) / rate
        else:
            remaining = 0

        print("%d/%d complete  %0.2f images/sec  %dm%ds elapsed  %dm%ds remaining" % (num_complete, total, rate, elapsed // 60, elapsed % 60, remaining // 60, remaining % 60))

        last_complete = now


def main():
    if not os.path.exists(a.output_dir):
        os.makedirs(a.output_dir)

    src_paths = []
    dst_paths = []

    for src_path in im.find(a.input_dir):
        name, _ = os.path.splitext(os.path.basename(src_path))
        dst_path = os.path.join(a.output_dir, name + ".png")
        if not os.path.exists(dst_path):
            src_paths.append(src_path)
            dst_paths.append(dst_path)
    
    global total
    total = len(src_paths)
    
    if a.workers == 1:
        
        with tf.compat.v1.Session() as sess:
            for src_path, dst_path in zip(src_paths, dst_paths):
                process(src_path, dst_path)
                complete()
    else:
        queue = tf.train.input_producer(zip(src_paths, dst_paths), shuffle=False, num_epochs=1)
        dequeue_op = queue.dequeue()

        def worker(coord):
            with sess.as_default():
                while not coord.should_stop():
                    try:
                        src_path, dst_path = sess.run(dequeue_op)
                    except tf.errors.OutOfRangeError:
                        coord.request_stop()
                        break

                    process(src_path, dst_path)
                    complete()

        # init epoch counter for the queue
        local_init_op = tf.local_variables_initializer()
        with tf.compat.v1.Session() as sess:
            sess.run(local_init_op)

            coord = tf.train.Coordinator()
            threads = tf.train.start_queue_runners(coord=coord)
            for i in range(a.workers):
                t = threading.Thread(target=worker, args=(coord,))
                t.start()
                threads.append(t)
            
            try:
                coord.join(threads)
            except KeyboardInterrupt:
                coord.request_stop()
                coord.join(threads)

main()

In [ ]:
# custom write the pix2pix to fix imshow issue in colab
%%writefile /content/installed_resp/piss-ant-pix2pix/pix2pix_custom.py
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow.compat.v1 as tf
import numpy as np
import argparse
import os
import json
import glob
import random
import collections
import math
import time
tf.disable_eager_execution()
parser = argparse.ArgumentParser()
parser.add_argument("--input_dir", help="path to folder containing images")
parser.add_argument("--mode", required=True, choices=["train", "test", "export"])
parser.add_argument("--output_dir", required=True, help="where to put output files")
parser.add_argument("--seed", type=int, default=0)
parser.add_argument("--checkpoint", default=None, help="directory with checkpoint to resume training from or use for testing")

parser.add_argument("--max_steps", type=int, help="number of training steps (0 to disable)")
parser.add_argument("--max_epochs", type=int, default=3, help="number of training epochs")
parser.add_argument("--summary_freq", type=int, default=100, help="update summaries every summary_freq steps")
parser.add_argument("--progress_freq", type=int, default=50, help="display progress every progress_freq steps")
parser.add_argument("--trace_freq", type=int, default=0, help="trace execution every trace_freq steps")
parser.add_argument("--display_freq", type=int, default=0, help="write current training images every display_freq steps")
parser.add_argument("--save_freq", type=int, default=100000, help="save model every save_freq steps, 0 to disable")

parser.add_argument("--aspect_ratio", type=float, default=1.0, help="aspect ratio of output images (width/height)")
parser.add_argument("--lab_colorization", action="store_true", help="split input image into brightness (A) and color (B)")
parser.add_argument("--batch_size", type=int, default=1, help="number of images in batch")
parser.add_argument("--which_direction", type=str, default="AtoB", choices=["AtoB", "BtoA"])
parser.add_argument("--ngf", type=int, default=64, help="number of generator filters in first conv layer")
parser.add_argument("--ndf", type=int, default=64, help="number of discriminator filters in first conv layer")
parser.add_argument("--scale_size", type=int, default=266, help="scale images to this size before cropping to 256x256")
parser.add_argument("--flip", dest="flip", action="store_true", help="flip images horizontally")
parser.add_argument("--no_flip", dest="flip", action="store_false", help="don't flip images horizontally")
parser.set_defaults(flip=True)
parser.add_argument("--lr", type=float, default=0.00005, help="initial learning rate")
parser.add_argument("--beta1", type=float, default=0.5, help="momentum term of adam")
parser.add_argument("--l1_weight", type=float, default=0.0, help="weight on L1 term for generator gradient")
parser.add_argument("--gan_weight", type=float, default=1.0, help="weight on GAN term for generator gradient")
parser.add_argument("--skip_connection", type=int, default= 1, help="@luyi: wether to use skip connection")
parser.add_argument("--patch_gan", type=int, default=1, help="@luyi: wether to use patch gan")
parser.add_argument("--wgan", type=int, default=1, help="@luyi: wether to use wgan")
a = parser.parse_args()

if a.wgan:
    clip = 0.01 #luyi clip discirminator in wgan
    n_critic = 5    #luyi loop discriminator
else:
    clip = 1e10
EPS = 1e-12
CROP_SIZE = 256

Examples = collections.namedtuple("Examples", "paths, inputs, targets, count, steps_per_epoch")
Model = collections.namedtuple("Model", "outputs, predict_real, predict_fake, dloss_GAN, discrim_grads_and_vars, gloss_GAN, gloss_L1, gen_grads_and_vars,\
 update_losses, incr_global_step, gen_train, discrim_train, gen_loss, discrim_loss, dloss_WGAN, gloss_WGAN")    #edited by luyi separate training operators


def preprocess(image):
    with tf.name_scope("preprocess"):
        # [0, 1] => [-1, 1]
        return image * 2 - 1


def deprocess(image):
    with tf.name_scope("deprocess"):
        # [-1, 1] => [0, 1]
        return (image + 1) / 2


def preprocess_lab(lab):
    with tf.name_scope("preprocess_lab"):
        L_chan, a_chan, b_chan = tf.unstack(lab, axis=2)
        # L_chan: black and white with input range [0, 100]
        # a_chan/b_chan: color channels with input range ~[-110, 110], not exact
        # [0, 100] => [-1, 1],  ~[-110, 110] => [-1, 1]
        return [L_chan / 50 - 1, a_chan / 110, b_chan / 110]


def deprocess_lab(L_chan, a_chan, b_chan):
    with tf.name_scope("deprocess_lab"):
        # this is axis=3 instead of axis=2 because we process individual images but deprocess batches
        return tf.stack([(L_chan + 1) / 2 * 100, a_chan * 110, b_chan * 110], axis=3)


def augment(image, brightness):
    # (a, b) color channels, combine with L channel and convert to rgb
    a_chan, b_chan = tf.unstack(image, axis=3)
    L_chan = tf.squeeze(brightness, axis=3)
    lab = deprocess_lab(L_chan, a_chan, b_chan)
    rgb = lab_to_rgb(lab)
    return rgb


def conv(batch_input, out_channels, stride):
    with tf.variable_scope("conv"):
        in_channels = batch_input.get_shape()[3]
        filter = tf.get_variable("filter", [4, 4, in_channels, out_channels], dtype=tf.float32, initializer=tf.random_normal_initializer(0, 0.02))
        # [batch, in_height, in_width, in_channels], [filter_width, filter_height, in_channels, out_channels]
        #     => [batch, out_height, out_width, out_channels]
        padded_input = tf.pad(batch_input, [[0, 0], [1, 1], [1, 1], [0, 0]], mode="CONSTANT")
        conv = tf.nn.conv2d(padded_input, filter, [1, stride, stride, 1], padding="VALID")
        return conv


def lrelu(x, a):
    with tf.name_scope("lrelu"):
        # adding these together creates the leak part and linear part
        # then cancels them out by subtracting/adding an absolute value term
        # leak: a*x/2 - a*abs(x)/2
        # linear: x/2 + abs(x)/2

        # this block looks like it has 2 inputs on the graph unless we do this
        x = tf.identity(x)
        return (0.5 * (1 + a)) * x + (0.5 * (1 - a)) * tf.abs(x)


def batchnorm(input):
    with tf.variable_scope("batchnorm"):
        # this block looks like it has 3 inputs on the graph unless we do this
        input = tf.identity(input)

        channels = input.get_shape()[3]
        offset = tf.get_variable("offset", [channels], dtype=tf.float32, initializer=tf.zeros_initializer())
        scale = tf.get_variable("scale", [channels], dtype=tf.float32, initializer=tf.random_normal_initializer(1.0, 0.02))
        mean, variance = tf.nn.moments(input, axes=[0, 1, 2], keep_dims=False)
        variance_epsilon = 1e-5
        normalized = tf.nn.batch_normalization(input, mean, variance, offset, scale, variance_epsilon=variance_epsilon)
        return normalized


def deconv(batch_input, out_channels):
    with tf.variable_scope("deconv"):
        batch, in_height, in_width, in_channels = [int(d) for d in batch_input.get_shape()]
        filter = tf.get_variable("filter", [4, 4, out_channels, in_channels], dtype=tf.float32, initializer=tf.random_normal_initializer(0, 0.02))
        # [batch, in_height, in_width, in_channels], [filter_width, filter_height, out_channels, in_channels]
        #     => [batch, out_height, out_width, out_channels]
        conv = tf.nn.conv2d_transpose(batch_input, filter, [batch, in_height * 2, in_width * 2, out_channels], [1, 2, 2, 1], padding="SAME")
        return conv


def check_image(image):
    assertion = tf.assert_equal(tf.shape(image)[-1], 3, message="image must have 3 color channels")
    assertion2 = tf.assert_less_equal(image, 1, message="image value <= 1")
    assertion3 = tf.assert_greater_equal(image, -1, message="image value >= -1")
    with tf.control_dependencies([assertion, assertion2, assertion3]):
        image = tf.identity(image)

    if image.get_shape().ndims not in (3, 4):
        raise ValueError("image must be either 3 or 4 dimensions")

    # make the last dimension 3 so that you can unstack the colors
    shape = list(image.get_shape())
    shape[-1] = 3
    image.set_shape(shape)
    return image

# based on https://github.com/torch/image/blob/9f65c30167b2048ecbe8b7befdc6b2d6d12baee9/generic/image.c
def rgb_to_lab(srgb):
    with tf.name_scope("rgb_to_lab"):
        srgb = check_image(srgb)
        srgb_pixels = tf.reshape(srgb, [-1, 3])

        with tf.name_scope("srgb_to_xyz"):
            linear_mask = tf.cast(srgb_pixels <= 0.04045, dtype=tf.float32)
            exponential_mask = tf.cast(srgb_pixels > 0.04045, dtype=tf.float32)
            rgb_pixels = (srgb_pixels / 12.92 * linear_mask) + (((srgb_pixels + 0.055) / 1.055) ** 2.4) * exponential_mask
            rgb_to_xyz = tf.constant([
                #    X        Y          Z
                [0.412453, 0.212671, 0.019334], # R
                [0.357580, 0.715160, 0.119193], # G
                [0.180423, 0.072169, 0.950227], # B
            ])
            xyz_pixels = tf.matmul(rgb_pixels, rgb_to_xyz)

        # https://en.wikipedia.org/wiki/Lab_color_space#CIELAB-CIEXYZ_conversions
        with tf.name_scope("xyz_to_cielab"):
            # convert to fx = f(X/Xn), fy = f(Y/Yn), fz = f(Z/Zn)

            # normalize for D65 white point
            xyz_normalized_pixels = tf.multiply(xyz_pixels, [1/0.950456, 1.0, 1/1.088754])

            epsilon = 6/29
            linear_mask = tf.cast(xyz_normalized_pixels <= (epsilon**3), dtype=tf.float32)
            exponential_mask = tf.cast(xyz_normalized_pixels > (epsilon**3), dtype=tf.float32)
            fxfyfz_pixels = (xyz_normalized_pixels / (3 * epsilon**2) + 4/29) * linear_mask + (xyz_normalized_pixels ** (1/3)) * exponential_mask

            # convert to lab
            fxfyfz_to_lab = tf.constant([
                #  l       a       b
                [  0.0,  500.0,    0.0], # fx
                [116.0, -500.0,  200.0], # fy
                [  0.0,    0.0, -200.0], # fz
            ])
            lab_pixels = tf.matmul(fxfyfz_pixels, fxfyfz_to_lab) + tf.constant([-16.0, 0.0, 0.0])

        return tf.reshape(lab_pixels, tf.shape(srgb))


def lab_to_rgb(lab):
    with tf.name_scope("lab_to_rgb"):
        lab = check_image(lab)
        lab_pixels = tf.reshape(lab, [-1, 3])

        # https://en.wikipedia.org/wiki/Lab_color_space#CIELAB-CIEXYZ_conversions
        with tf.name_scope("cielab_to_xyz"):
            # convert to fxfyfz
            lab_to_fxfyfz = tf.constant([
                #   fx      fy        fz
                [1/116.0, 1/116.0,  1/116.0], # l
                [1/500.0,     0.0,      0.0], # a
                [    0.0,     0.0, -1/200.0], # b
            ])
            fxfyfz_pixels = tf.matmul(lab_pixels + tf.constant([16.0, 0.0, 0.0]), lab_to_fxfyfz)

            # convert to xyz
            epsilon = 6/29
            linear_mask = tf.cast(fxfyfz_pixels <= epsilon, dtype=tf.float32)
            exponential_mask = tf.cast(fxfyfz_pixels > epsilon, dtype=tf.float32)
            xyz_pixels = (3 * epsilon**2 * (fxfyfz_pixels - 4/29)) * linear_mask + (fxfyfz_pixels ** 3) * exponential_mask

            # denormalize for D65 white point
            xyz_pixels = tf.multiply(xyz_pixels, [0.950456, 1.0, 1.088754])

        with tf.name_scope("xyz_to_srgb"):
            xyz_to_rgb = tf.constant([
                #     r           g          b
                [ 3.2404542, -0.9692660,  0.0556434], # x
                [-1.5371385,  1.8760108, -0.2040259], # y
                [-0.4985314,  0.0415560,  1.0572252], # z
            ])
            rgb_pixels = tf.matmul(xyz_pixels, xyz_to_rgb)
            # avoid a slightly negative number messing up the conversion
            rgb_pixels = tf.clip_by_value(rgb_pixels, 0.0, 1.0)
            linear_mask = tf.cast(rgb_pixels <= 0.0031308, dtype=tf.float32)
            exponential_mask = tf.cast(rgb_pixels > 0.0031308, dtype=tf.float32)
            srgb_pixels = (rgb_pixels * 12.92 * linear_mask) + ((rgb_pixels ** (1/2.4) * 1.055) - 0.055) * exponential_mask

        return tf.reshape(srgb_pixels, tf.shape(lab))


def load_examples():
    print(a.input_dir)
    if a.input_dir is None or not os.path.exists(a.input_dir):
        raise Exception("input_dir does not exist")

    input_paths = glob.glob(os.path.join(a.input_dir, "*.jpg"))
    decode = tf.image.decode_jpeg
    if len(input_paths) == 0:
        input_paths = glob.glob(os.path.join(a.input_dir, "*.png"))
        decode = tf.image.decode_png

    if len(input_paths) == 0:
        raise Exception("input_dir contains no image files")

    def get_name(path):
        name, _ = os.path.splitext(os.path.basename(path))
        return name

    # if the image names are numbers, sort by the value rather than asciibetically
    # having sorted inputs means that the outputs are sorted in test mode
    if all(get_name(path).isdigit() for path in input_paths):
        input_paths = sorted(input_paths, key=lambda path: int(get_name(path)))
    else:
        input_paths = sorted(input_paths)

    with tf.name_scope("load_images"):
        path_queue = tf.train.string_input_producer(input_paths, shuffle=a.mode == "train")
        reader = tf.WholeFileReader()
        paths, contents = reader.read(path_queue)
        raw_input = decode(contents)
        raw_input = tf.image.convert_image_dtype(raw_input, dtype=tf.float32)

        assertion = tf.assert_equal(tf.shape(raw_input)[2], 3, message="image does not have 3 channels")
        with tf.control_dependencies([assertion]):
            raw_input = tf.identity(raw_input)

        raw_input.set_shape([None, None, 3])

        if a.lab_colorization:
            # load color and brightness from image, no B image exists here
            lab = rgb_to_lab(raw_input)
            L_chan, a_chan, b_chan = preprocess_lab(lab)
            a_images = tf.expand_dims(L_chan, axis=2)
            b_images = tf.stack([a_chan, b_chan], axis=2)
        else:
            # break apart image pair and move to range [-1, 1]
            width = tf.shape(raw_input)[1] # [height, width, channels]
            a_images = preprocess(raw_input[:,:width//2,:])
            b_images = preprocess(raw_input[:,width//2:,:])


    if a.which_direction == "AtoB":
        inputs, targets = [a_images, b_images]
    elif a.which_direction == "BtoA":
        inputs, targets = [b_images, a_images]
    else:
        raise Exception("invalid direction")

    # synchronize seed for image operations so that we do the same operations to both
    # input and output images
    seed = random.randint(0, 2**31 - 1)
    def transform(image):
        r = image
        if a.flip:
            r = tf.image.random_flip_left_right(r, seed=seed)

        # area produces a nice downscaling, but does nearest neighbor for upscaling
        # assume we're going to be doing downscaling here
        r = tf.image.resize_images(r, [a.scale_size, a.scale_size], method=tf.image.ResizeMethod.AREA)

        offset = tf.cast(tf.floor(tf.random_uniform([2], 0, a.scale_size - CROP_SIZE + 1, seed=seed)), dtype=tf.int32)
        if a.scale_size > CROP_SIZE:
            r = tf.image.crop_to_bounding_box(r, offset[0], offset[1], CROP_SIZE, CROP_SIZE)
        elif a.scale_size < CROP_SIZE:
            raise Exception("scale size cannot be less than crop size")
        return r

    with tf.name_scope("input_images"):
        input_images = transform(inputs)

    with tf.name_scope("target_images"):
        target_images = transform(targets)

    paths_batch, inputs_batch, targets_batch = tf.train.batch([paths, input_images, target_images], batch_size=a.batch_size)
    steps_per_epoch = int(math.ceil(len(input_paths) / a.batch_size))

    return Examples(
        paths=paths_batch,
        inputs=inputs_batch,
        targets=targets_batch,
        count=len(input_paths),
        steps_per_epoch=steps_per_epoch,
    )


def create_generator(generator_inputs, generator_outputs_channels):
    layers = []

    # encoder_1: [batch, 256, 256, in_channels] => [batch, 128, 128, ngf]
    with tf.variable_scope("encoder_1"):
        output = conv(generator_inputs, a.ngf, stride=2)
        layers.append(output)

    layer_specs = [
        a.ngf * 2, # encoder_2: [batch, 128, 128, ngf] => [batch, 64, 64, ngf * 2]
        a.ngf * 4, # encoder_3: [batch, 64, 64, ngf * 2] => [batch, 32, 32, ngf * 4]
        a.ngf * 8, # encoder_4: [batch, 32, 32, ngf * 4] => [batch, 16, 16, ngf * 8]
        a.ngf * 8, # encoder_5: [batch, 16, 16, ngf * 8] => [batch, 8, 8, ngf * 8]
        a.ngf * 8, # encoder_6: [batch, 8, 8, ngf * 8] => [batch, 4, 4, ngf * 8]
        a.ngf * 8, # encoder_7: [batch, 4, 4, ngf * 8] => [batch, 2, 2, ngf * 8]
        a.ngf * 8, # encoder_8: [batch, 2, 2, ngf * 8] => [batch, 1, 1, ngf * 8]
    ]

    for out_channels in layer_specs:
        with tf.variable_scope("encoder_%d" % (len(layers) + 1)):
            rectified = lrelu(layers[-1], 0.2)
            # [batch, in_height, in_width, in_channels] => [batch, in_height/2, in_width/2, out_channels]
            convolved = conv(rectified, out_channels, stride=2)
            output = batchnorm(convolved)
            layers.append(output)

    layer_specs = [
        (a.ngf * 8, 0.5),   # decoder_8: [batch, 1, 1, ngf * 8] => [batch, 2, 2, ngf * 8 * 2]
        (a.ngf * 8, 0.5),   # decoder_7: [batch, 2, 2, ngf * 8 * 2] => [batch, 4, 4, ngf * 8 * 2]
        (a.ngf * 8, 0.5),   # decoder_6: [batch, 4, 4, ngf * 8 * 2] => [batch, 8, 8, ngf * 8 * 2]
        (a.ngf * 8, 0.0),   # decoder_5: [batch, 8, 8, ngf * 8 * 2] => [batch, 16, 16, ngf * 8 * 2]
        (a.ngf * 4, 0.0),   # decoder_4: [batch, 16, 16, ngf * 8 * 2] => [batch, 32, 32, ngf * 4 * 2]
        (a.ngf * 2, 0.0),   # decoder_3: [batch, 32, 32, ngf * 4 * 2] => [batch, 64, 64, ngf * 2 * 2]
        (a.ngf, 0.0),       # decoder_2: [batch, 64, 64, ngf * 2 * 2] => [batch, 128, 128, ngf * 2]
    ]

    num_encoder_layers = len(layers)
    for decoder_layer, (out_channels, dropout) in enumerate(layer_specs):
        skip_layer = num_encoder_layers - decoder_layer - 1 #skip connection to remove?
        with tf.variable_scope("decoder_%d" % (num_encoder_layers - decoder_layer)):
            if decoder_layer == 0:
                # first decoder layer doesn't have skip connections
                # since it is directly connected to the skip_layer
                input = layers[-1]
            else:
                if a.skip_connection:
                    input = tf.concat([layers[-1], layers[skip_layer]], axis=3)
                else:
                    input = layers[-1]  #luyi try to remove skip connection

            rectified = tf.nn.relu(input)
            # [batch, in_height, in_width, in_channels] => [batch, in_height*2, in_width*2, out_channels]
            output = deconv(rectified, out_channels)
            output = batchnorm(output)

            if dropout > 0.0:
                output = tf.nn.dropout(output, keep_prob=1 - dropout)

            layers.append(output)

    # decoder_1: [batch, 128, 128, ngf * 2] => [batch, 256, 256, generator_outputs_channels]
    with tf.variable_scope("decoder_1"):
        if a.skip_connection:
            input = tf.concat([layers[-1], layers[0]], axis=3) #skip connection seems essential for colorization
        else:
            input = layers[-1]  #luyi remove skip connection
        rectified = tf.nn.relu(input)
        output = deconv(rectified, generator_outputs_channels)
        output = tf.tanh(output)
        layers.append(output)
    return layers[-1]

def create_discriminator(discrim_inputs, discrim_targets):
    n_layers = 3
    layers = []

    # 2x [batch, height, width, in_channels] => [batch, height, width, in_channels * 2]
    input = tf.concat([discrim_inputs, discrim_targets], axis=3)

    # layer_1: [batch, 256, 256, in_channels * 2] => [batch, 128, 128, ndf]
    with tf.variable_scope("layer_1"):
        convolved = conv(input, a.ndf, stride=2)
        rectified = lrelu(convolved, 0.2)
        layers.append(rectified)

    # layer_2: [batch, 128, 128, ndf] => [batch, 64, 64, ndf * 2]
    # layer_3: [batch, 64, 64, ndf * 2] => [batch, 32, 32, ndf * 4]
    # layer_4: [batch, 32, 32, ndf * 4] => [batch, 31, 31, ndf * 8]
    for i in range(n_layers):
        with tf.variable_scope("layer_%d" % (len(layers) + 1)):
            out_channels = a.ndf * min(2**(i+1), 8)
            stride = 1 if i == n_layers - 1 else 2  # last layer here has stride 1
            convolved = conv(layers[-1], out_channels, stride=stride)
            normalized = batchnorm(convolved)
            rectified = lrelu(normalized, 0.2)
            layers.append(rectified)

    # layer_5: [batch, 31, 31, ndf * 8] => [batch, 30, 30, 1]
    with tf.variable_scope("layer_%d" % (len(layers) + 1)):
        convolved = conv(rectified, out_channels=1, stride=1)
        if a.patch_gan: #luyi patch wgan
            output = convolved
        else:
            print("--------PATCH GAN OFF--------")
            batch_size, poolh, poolw, channel = convolved.get_shape().as_list() #luyi patch gan removed
            flat_convolved = tf.reshape(convolved, [batch_size, -1])
            W = tf.Variable(tf.truncated_normal([poolh*poolw,1], stddev=0.1), name="W")
            b = tf.Variable(tf.constant(0.1, shape=[1]), name="b")
            logit = tf.nn.xw_plus_b(flat_convolved, W, b)
            output = logit  #luyi wgan
        layers.append(output)
    return layers[-1]


def create_model(inputs, targets):
    with tf.variable_scope("generator") as scope:
        out_channels = int(targets.get_shape()[-1])
        outputs = create_generator(inputs, out_channels)

    # create two copies of discriminator, one for real pairs and one for fake pairs
    # they share the same underlying variables
    with tf.name_scope("real_discriminator"):
        with tf.variable_scope("discriminator"):
            # 2x [batch, height, width, channels] => [batch, 30, 30, 1]
            predict_real = create_discriminator(inputs, targets)

    with tf.name_scope("fake_discriminator"):
        with tf.variable_scope("discriminator", reuse=True):
            # 2x [batch, height, width, channels] => [batch, 30, 30, 1]
            predict_fake = create_discriminator(inputs, outputs)

    with tf.name_scope("discriminator_loss"):
        # minimizing -tf.log will try to get inputs to 1
        # predict_real => 1
        # predict_fake => 0
        dloss_GAN = tf.reduce_mean(-(tf.log(tf.sigmoid(predict_real) + EPS) + tf.log(1 - tf.sigmoid(predict_fake) + EPS)))
        dloss_WGAN = tf.reduce_mean(predict_fake - predict_real)  #luyi wgan critic loss
        discrim_loss = tf.identity(dloss_WGAN) if a.wgan else tf.identity(dloss_GAN)  #discriminator loss either from wgan or gan

    with tf.name_scope("generator_loss"):
        # predict_fake => 1
        # abs(targets - outputs) => 0
        gloss_GAN = tf.reduce_mean(-tf.log(tf.sigmoid(predict_fake) + EPS))
        gloss_WGAN = tf.reduce_mean(-predict_fake)    #luyi wgan generator loss
        gen_loss = tf.identity(gloss_WGAN) if a.wgan else tf.identity(gloss_GAN)  #luyi generator loss either from wgan or gan
        gloss_L1 = tf.reduce_mean(tf.abs(targets - outputs))

    with tf.name_scope("discriminator_train"):
        discrim_tvars = [var for var in tf.trainable_variables() if var.name.startswith("discriminator")]
        print('Discriminator Variables:')
        for var in discrim_tvars:
            print(var.name)
        if not a.wgan:
            discrim_optim = tf.train.AdamOptimizer(a.lr, a.beta1)
        else:
            discrim_optim = tf.train.RMSPropOptimizer(a.lr)
        clipped_var = [tf.assign(var, tf.clip_by_value(var, -clip, clip)) for var in discrim_tvars] #luyi wgan clip discriminator variables
        with tf.control_dependencies(clipped_var):  #luyi clip variables first
            discrim_grads_and_vars = discrim_optim.compute_gradients(discrim_loss, var_list=discrim_tvars)
            discrim_train = discrim_optim.apply_gradients(discrim_grads_and_vars)

    with tf.name_scope("generator_train"):
        #with tf.control_dependencies([discrim_train]):
        gen_tvars = [var for var in tf.trainable_variables() if var.name.startswith("generator")]
        print('Generator Variables:')
        for var in gen_tvars:
            print(var.name)
        if not a.wgan:
            gen_optim = tf.train.AdamOptimizer(a.lr, a.beta1)
        else:
            gen_optim = tf.train.RMSPropOptimizer(a.lr) #luyi optimizer
        gen_grads_and_vars = gen_optim.compute_gradients(gen_loss * a.gan_weight + gloss_L1 * a.l1_weight, var_list=gen_tvars)
        gen_train = gen_optim.apply_gradients(gen_grads_and_vars)

    ema = tf.train.ExponentialMovingAverage(decay=0.99)
    update_losses = ema.apply([dloss_WGAN, gloss_WGAN, dloss_GAN, gloss_GAN, gloss_L1])

    global_step = tf.compat.v1.train.get_or_create_global_step()
    incr_global_step = tf.assign(global_step, global_step+1)

    return Model(
        predict_real=predict_real,
        predict_fake=predict_fake,
        dloss_GAN=ema.average(dloss_GAN),
        dloss_WGAN=ema.average(dloss_WGAN),
        discrim_grads_and_vars=discrim_grads_and_vars,
        gloss_GAN=ema.average(gloss_GAN),
        gloss_WGAN=ema.average(gloss_WGAN),
        gloss_L1=ema.average(gloss_L1),
        gen_grads_and_vars=gen_grads_and_vars,
        outputs=outputs,
        update_losses=update_losses,
        gen_train=gen_train,
        discrim_train=discrim_train,
        incr_global_step=incr_global_step,
        gen_loss=gen_loss,
        discrim_loss=discrim_loss
    )


def save_images(fetches, step=None):
    image_dir = os.path.join(a.output_dir, "images")
    if not os.path.exists(image_dir):
        os.makedirs(image_dir)

    filesets = []
    for i, in_path in enumerate(fetches["paths"]):
        name, _ = os.path.splitext(os.path.basename(in_path.decode("utf8")))
        fileset = {"name": name, "step": step}
        for kind in ["inputs", "outputs", "targets"]:
            filename = name + "-" + kind + ".png"
            if step is not None:
                filename = "%08d-%s" % (step, filename)
            fileset[kind] = filename
            out_path = os.path.join(image_dir, filename)
            contents = fetches[kind][i]
            with open(out_path, "wb") as f:
                f.write(contents)
        filesets.append(fileset)
    return filesets


def append_index(filesets, step=False):
    index_path = os.path.join(a.output_dir, "index.html")
    if os.path.exists(index_path):
        index = open(index_path, "a")
    else:
        index = open(index_path, "w")
        index.write("<html><body><table><tr>")
        if step:
            index.write("<th>step</th>")
        index.write("<th>name</th><th>input</th><th>output</th><th>target</th></tr>")

    for fileset in filesets:
        index.write("<tr>")

        if step:
            index.write("<td>%d</td>" % fileset["step"])
        index.write("<td>%s</td>" % fileset["name"])

        for kind in ["inputs", "outputs", "targets"]:
            index.write("<td><img src='images/%s'></td>" % fileset[kind])

        index.write("</tr>")
    return index_path


def main():
    #if tf.__version__ != "1.0.0":
    #    raise Exception("Tensorflow version 1.0.0 required")

    if a.seed is None:
        a.seed = random.randint(0, 2**31 - 1)
    #tf.random.set_seed(a.seed)
    tf.set_random_seed(a.seed)
    np.random.seed(a.seed)
    random.seed(a.seed)

    if not os.path.exists(a.output_dir):
        os.makedirs(a.output_dir)

    if a.mode == "test" or a.mode == "export":
        if a.checkpoint is None:
            raise Exception("checkpoint required for test mode")

        # load some options from the checkpoint
        options = {"which_direction", "ngf", "ndf", "lab_colorization"}
        with open(os.path.join(a.checkpoint, "options.json")) as f:
            for key, val in json.loads(f.read()).items():
                if key in options:
                    print("loaded", key, "=", val)
                    setattr(a, key, val)
        # disable these features in test mode
        a.scale_size = CROP_SIZE
        a.flip = False

    for k, v in a._get_kwargs():
        print(k, "=", v)

    with open(os.path.join(a.output_dir, "options.json"), "w") as f:
        f.write(json.dumps(vars(a), sort_keys=True, indent=4))

    if a.mode == "export":
        # export the generator to a meta graph that can be imported later for standalone generation
        if a.lab_colorization:
            raise Exception("export not supported for lab_colorization")

        input = tf.placeholder(tf.string, shape=[1])
        input_data = tf.decode_base64(input[0])
        input_image = tf.image.decode_png(input_data)
        # remove alpha channel if present
        input_image = input_image[:,:,:3]
        input_image = tf.image.convert_image_dtype(input_image, dtype=tf.float32)
        input_image.set_shape([CROP_SIZE, CROP_SIZE, 3])
        batch_input = tf.expand_dims(input_image, axis=0)

        with tf.variable_scope("generator") as scope:
            batch_output = deprocess(create_generator(preprocess(batch_input), 3))

        output_image = tf.image.convert_image_dtype(batch_output, dtype=tf.uint8, saturate=True)[0]
        output_data = tf.image.encode_png(output_image)
        output = tf.convert_to_tensor([tf.encode_base64(output_data)])

        key = tf.placeholder(tf.string, shape=[1])
        inputs = {
            "key": key.name,
            "input": input.name
        }
        tf.add_to_collection("inputs", json.dumps(inputs))
        outputs = {
            "key":  tf.identity(key).name,
            "output": output.name,
        }
        tf.add_to_collection("outputs", json.dumps(outputs))

        init_op = tf.global_variables_initializer()
        restore_saver = tf.train.Saver()
        export_saver = tf.train.Saver()

        with tf.Session() as sess:
            sess.run(init_op)
            print("loading model from checkpoint")
            checkpoint = tf.train.latest_checkpoint(a.checkpoint)
            restore_saver.restore(sess, checkpoint)
            print("exporting model")
            export_saver.export_meta_graph(filename=os.path.join(a.output_dir, "export.meta"))
            export_saver.save(sess, os.path.join(a.output_dir, "export"), write_meta_graph=False)

        return

    examples = load_examples()
    print("examples count = %d" % examples.count)

    # inputs and targets are [batch_size, height, width, channels]
    model = create_model(examples.inputs, examples.targets)

    # undo colorization splitting on images that we use for display/output
    if a.lab_colorization:
        if a.which_direction == "AtoB":
            # inputs is brightness, this will be handled fine as a grayscale image
            # need to augment targets and outputs with brightness
            targets = augment(examples.targets, examples.inputs)
            outputs = augment(model.outputs, examples.inputs)
            # inputs can be deprocessed normally and handled as if they are single channel
            # grayscale images
            inputs = deprocess(examples.inputs)
        elif a.which_direction == "BtoA":
            # inputs will be color channels only, get brightness from targets
            inputs = augment(examples.inputs, examples.targets)
            targets = deprocess(examples.targets)
            outputs = deprocess(model.outputs)
        else:
            raise Exception("invalid direction")
    else:
        inputs = deprocess(examples.inputs)
        targets = deprocess(examples.targets)
        outputs = deprocess(model.outputs)

    def convert(image):
        if a.aspect_ratio != 1.0:
            # upscale to correct aspect ratio
            size = [CROP_SIZE, int(round(CROP_SIZE * a.aspect_ratio))]
            image = tf.image.resize_images(image, size=size, method=tf.image.ResizeMethod.BICUBIC)

        return tf.image.convert_image_dtype(image, dtype=tf.uint8, saturate=True)

    # reverse any processing on images so they can be written to disk or displayed to user
    with tf.name_scope("convert_inputs"):
        converted_inputs = convert(inputs)

    with tf.name_scope("convert_targets"):
        converted_targets = convert(targets)

    with tf.name_scope("convert_outputs"):
        converted_outputs = convert(outputs)

    with tf.name_scope("encode_images"):
        display_fetches = {
            "paths": examples.paths,
            "inputs": tf.map_fn(tf.image.encode_png, converted_inputs, dtype=tf.string, name="input_pngs"),
            "targets": tf.map_fn(tf.image.encode_png, converted_targets, dtype=tf.string, name="target_pngs"),
            "outputs": tf.map_fn(tf.image.encode_png, converted_outputs, dtype=tf.string, name="output_pngs"),
        }

    # summaries
    with tf.name_scope("inputs_summary"):
        tf.summary.image("inputs", converted_inputs)

    with tf.name_scope("targets_summary"):
        tf.summary.image("targets", converted_targets)

    with tf.name_scope("outputs_summary"):
        tf.summary.image("outputs", converted_outputs)

    #with tf.name_scope("predict_real_summary"):
    #    tf.summary.image("predict_real", tf.image.convert_image_dtype(model.predict_real, dtype=tf.uint8))

    #with tf.name_scope("predict_fake_summary"):
    #    tf.summary.image("predict_fake", tf.image.convert_image_dtype(model.predict_fake, dtype=tf.uint8))

    tf.summary.scalar("dloss_GAN", model.dloss_GAN)
    tf.summary.scalar("gloss_GAN", model.gloss_GAN)
    tf.summary.scalar("gloss_L1", model.gloss_L1)
    if a.wgan:
        tf.summary.scalar("discrim_loss_WGAN", model.dloss_WGAN)
        tf.summary.scalar("gen_loss_WGAN", model.gloss_WGAN)

    for var in tf.trainable_variables():
        tf.summary.histogram(var.op.name + "/values", var)

    for grad, var in model.discrim_grads_and_vars + model.gen_grads_and_vars:
        tf.summary.histogram(var.op.name + "/gradients", grad)

    with tf.name_scope("parameter_count"):
        parameter_count = tf.reduce_sum([tf.reduce_prod(tf.shape(v)) for v in tf.trainable_variables()])

    saver = tf.train.Saver(max_to_keep=1)

    logdir = a.output_dir if (a.trace_freq > 0 or a.summary_freq > 0) else None
    sv = tf.train.Supervisor(logdir=logdir, save_summaries_secs=0, saver=None)
    with sv.managed_session() as sess:
        print("parameter_count =", sess.run(parameter_count))

        if a.checkpoint is not None:
            print("loading model from checkpoint")
            checkpoint = tf.train.latest_checkpoint(a.checkpoint)
            saver.restore(sess, checkpoint)

        max_steps = 2**32
        if a.max_epochs is not None:
            max_steps = examples.steps_per_epoch * a.max_epochs
        if a.max_steps is not None:
            max_steps = a.max_steps

        if a.mode == "test":
            # testing
            # at most, process the test data once
            max_steps = min(examples.steps_per_epoch, max_steps)
            for step in range(max_steps):
                results = sess.run(display_fetches)
                filesets = save_images(results)
                for i, f in enumerate(filesets):
                    print("evaluated image", f["name"])
                index_path = append_index(filesets)

            print("wrote index at", index_path)
        else:
            # training
            start = time.time()
            for step in range(max_steps):
                def should(freq):
                    return freq > 0 and ((step + 1) % freq == 0 or step == max_steps - 1)

                options = None
                run_metadata = None
                if should(a.trace_freq):
                    options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
                    run_metadata = tf.RunMetadata()

                fetches = {
                    "global_step": sv.global_step,
                }

                if should(a.progress_freq):
                    fetches["dloss_GAN"] = model.dloss_GAN
                    fetches["gloss_GAN"] = model.gloss_GAN
                    fetches["gloss_L1"] = model.gloss_L1

                if should(a.summary_freq):
                    fetches["summary"] = sv.summary_op

                if should(a.display_freq):
                    fetches["display"] = display_fetches

                if a.wgan:   #luyi trainning for wgan
                    fetches["update_losses"] = model.update_losses
                    for _ in range(n_critic):
                        _ = sess.run(model.discrim_train, options=options, run_metadata=run_metadata)
                    fetches["gen_train"] = model.gen_train
                    fetches["incr_global_step"] = model.incr_global_step
                    results = sess.run(fetches, options=options, run_metadata=run_metadata)
                else:
                    fetches["update_losses"] = model.update_losses
                    fetches["gen_train"]=model.gen_train
                    fetches["discrim_train"]=model.discrim_train
                    fetches["incr_global_step"]=model.incr_global_step
                    results = sess.run(fetches, options=options, run_metadata=run_metadata)

                if should(a.summary_freq):
                    print("recording summary")
                    sv.summary_writer.add_summary(results["summary"], results["global_step"])

                if should(a.display_freq):
                    print("saving display images")
                    filesets = save_images(results["display"], step=results["global_step"])
                    append_index(filesets, step=True)

                if should(a.trace_freq):
                    print("recording trace")
                    sv.summary_writer.add_run_metadata(run_metadata, "step_%d" % results["global_step"])

                if should(a.progress_freq):
                    # global_step will have the correct step count if we resume from a checkpoint
                    train_epoch = math.ceil(results["global_step"] / examples.steps_per_epoch)
                    train_step = (results["global_step"] - 1) % examples.steps_per_epoch + 1
                    rate = (step + 1) * a.batch_size / (time.time() - start)
                    remaining = (max_steps - step) * a.batch_size / rate
                    print("progress  epoch %d  step %d  image/sec %0.1f  remaining %dm" % (train_epoch, train_step, rate, remaining / 60))
                    print("dloss_GAN", results["dloss_GAN"])
                    print("gloss_GAN", results["gloss_GAN"])
                    print("gloss_L1", results["gloss_L1"])

                if should(a.save_freq):
                    print("saving model")
                    saver.save(sess, os.path.join(a.output_dir, "model"), global_step=sv.global_step)

                if sv.should_stop():
                    break


main()


In [ ]:
# custom write the edge tool to fix imshow issue in colab
%%writefile /content/installed_resp/piss-ant-pix2pix/tools/tfimage_custom.py
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import os
tf.compat.v1.disable_eager_execution()

def create_op(func, **placeholders):
    op = func(**placeholders)

    def f(**kwargs):
        feed_dict = {}
        for argname, argvalue in kwargs.items():
            placeholder = placeholders[argname]
            feed_dict[placeholder] = argvalue
        return tf.get_default_session().run(op, feed_dict=feed_dict)

    return f

downscale = create_op(
    func=tf.image.resize,
    images=tf.placeholder(tf.float32, [None, None, None]),
    size=tf.placeholder(tf.int32, [2]),
    method=tf.image.ResizeMethod.AREA,
)

upscale = create_op(
    func=tf.image.resize,
    images=tf.placeholder(tf.float32, [None, None, None]),
    size=tf.placeholder(tf.int32, [2]),
    method=tf.image.ResizeMethod.BICUBIC,
)

decode_jpeg = create_op(
    func=tf.image.decode_jpeg,
    contents=tf.placeholder(tf.string),
)

decode_png = create_op(
    func=tf.image.decode_png,
    contents=tf.placeholder(tf.string),
)

rgb_to_grayscale = create_op(
    func=tf.image.rgb_to_grayscale,
    images=tf.placeholder(tf.float32),
)

grayscale_to_rgb = create_op(
    func=tf.image.grayscale_to_rgb,
    images=tf.placeholder(tf.float32),
)

encode_jpeg = create_op(
    func=tf.image.encode_jpeg,
    image=tf.placeholder(tf.uint8),
)

encode_png = create_op(
    func=tf.image.encode_png,
    image=tf.placeholder(tf.uint8),
)

crop = create_op(
    func=tf.image.crop_to_bounding_box,
    image=tf.placeholder(tf.float32),
    offset_height=tf.placeholder(tf.int32, []),
    offset_width=tf.placeholder(tf.int32, []),
    target_height=tf.placeholder(tf.int32, []),
    target_width=tf.placeholder(tf.int32, []),
)

pad = create_op(
    func=tf.image.pad_to_bounding_box,
    image=tf.placeholder(tf.float32),
    offset_height=tf.placeholder(tf.int32, []),
    offset_width=tf.placeholder(tf.int32, []),
    target_height=tf.placeholder(tf.int32, []),
    target_width=tf.placeholder(tf.int32, []),
)

to_uint8 = create_op(
    func=tf.image.convert_image_dtype,
    image=tf.placeholder(tf.float32),
    dtype=tf.uint8,
    saturate=True,
)

to_float32 = create_op(
    func=tf.image.convert_image_dtype,
    image=tf.placeholder(tf.uint8),
    dtype=tf.float32,
)


def load(path):
    with open(path, 'rb') as f:
        contents = f.read()
        
    _, ext = os.path.splitext(path.lower())

    if ext == ".jpg":
        image = decode_jpeg(contents=contents)
    elif ext == ".png":
        image = decode_png(contents=contents)
    else:
        raise Exception("invalid image suffix")

    return to_float32(image=image)


def find(d):
    result = []
    for filename in os.listdir(d):
        _, ext = os.path.splitext(filename.lower())
        if ext == ".jpg" or ext == ".png":
            result.append(os.path.join(d, filename))
    result.sort()
    return result


def save(image, path, replace=False):
    _, ext = os.path.splitext(path.lower())
    image = to_uint8(image=image)
    if ext == ".jpg":
        encoded = encode_jpeg(image=image)
    elif ext == ".png":
        encoded = encode_png(image=image)
    else:
        raise Exception("invalid image suffix")

    dirname = os.path.dirname(path)
    if dirname != "" and not os.path.exists(dirname):
        os.makedirs(dirname)

    if os.path.exists(path):
        if replace:
            os.remove(path)
        else:
            raise Exception("file already exists at " + path)

    with open(path, "wb") as f:
        f.write(encoded)


In [ ]:
# custom write the edge tool to fix imshow issue in colab
%%writefile /content/installed_resp/piss-ant-pix2pix/tools/edge_custom.py
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import argparse
import cv2,sys,os
import numpy as np
from matplotlib import pyplot as plt
# p edge.py --input_dir  --output_dir
parser = argparse.ArgumentParser()
parser.add_argument("--input_dir", required=True, help="path to folder containing images")
parser.add_argument("--output_dir", required=True, help="output path")
a = parser.parse_args()




for img_path in os.listdir(a.input_dir):
	print(a.input_dir+'/'+img_path)

	# img = cv2.imread(a.input_dir+'/'+img_path,0)
	# print(a.input_dir+img_path)
	img = cv2.imread(a.input_dir+'/'+img_path, cv2.IMREAD_GRAYSCALE)
	# img = cv2.GaussianBlur(img, (11, 11), 0)
	# sobelx = cv2.Sobel(img, cv2.CV_64F, 1, 0)
	# sobely = cv2.Sobel(img, cv2.CV_64F, 0, 1)
	laplacian = cv2.Laplacian(img, cv2.CV_64F, ksize=5)
	# canny = cv2.Canny(img, 100, 150)
	# cv2.imshow("Image", img)
	# cv2.imshow("Sobelx", sobelx)
	# cv2.imshow("Sobely", sobely)
# 	cv2.imshow("Laplacian", laplacian)
	# cv2.imshow("Canny", canny)

	# # Converting the image to grayscale.
	# gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

	# # Using the Canny filter to get contours
	# edges = cv2.Canny(gray, 20, 30)
	# # Using the Canny filter with different parameters
	# # Stacking the images to print them together
	# # For comparison
	# images = np.hstack((gray, edges, edges_high_thresh))

	# # Display the resulting frame
	# cv2.imshow('Frame', canny_images)



	edges_high_thresh = cv2.Canny(img, 60, 120)
	edges = cv2.Canny(img,100,200)
	# cv2.imshow("edges",edges)
	# img_not = cv2.bitwise_not(edges)
	img_not_high = cv2.bitwise_not(edges_high_thresh)
	# cv2.imshow("Invert1",img_not)
	# plt.subplot(121),plt.imshow(img,cmap = 'gray')
	# plt.title('Original Image'), plt.xticks([]), plt.yticks([])
	# plt.imshow(img_not,cmap = 'gray')
# 	plt.imshow(img_not_high,cmap = 'gray')
	# plt.title('Edge Image'), plt.xticks([]), plt.yticks([])
	# Negative
	# write
	# cv2.imshow("Invert1",edges_neg)
	outfile=a.output_dir+'/'+img_path
	print(outfile)
	cv2.imwrite(outfile, img_not_high)
	# plt.show()





##Get images and make the dataset

## run it to connect your google drive

In [ ]:
# get  images
from google.colab import drive
drive.mount('/content/drive')
# clear_output()

# Paths to make

In [ ]:
sourceFolder = '/content/drive/MyDrive/Colab Notebooks/datasets/garbage'
#sourceFolder = '/content/drive/MyDrive/Colab Notebooks/datasets/hotwheelsWithNameAndYear' # fetch images recursive to make a dataset from this root folder
sourcepathSplit = sourceFolder.split('/') # split of dataset name 
trainoutputroot = sourcepathSplit[len(sourcepathSplit) - 1] # set  as outputroot
destFolder = '/content/installed_resp/piss-ant-pix2pix/'+  trainoutputroot  + '/png_input_images'
train_metrics_output_folder='/content/installed_resp/piss-ant-pix2pix/' + trainoutputroot + '/training_out'
datasetFolder = '/content/installed_resp/piss-ant-pix2pix/'+  trainoutputroot  +'/dataset_from_input'
train_input_folder = datasetFolder + '/_combined/train'
validate_input_folder = datasetFolder +  '/_combined/val'
validate_output_images = '/content/installed_resp/piss-ant-pix2pix/' + trainoutputroot + '/tested_images'



In [ ]:
ImgFoldersToMake= [
    '/content/installed_resp/piss-ant-pix2pix/'+  trainoutputroot ,
    '/content/installed_resp/piss-ant-pix2pix/'+  trainoutputroot  +'/png_input_images', # folder where png file come after renaming from jpg
    '/content/installed_resp/piss-ant-pix2pix/'+  trainoutputroot  +'/dataset_from_input', # outfolder for final preprocessed images
    '/content/installed_resp/piss-ant-pix2pix/' + trainoutputroot + '/training_out',
    '/content/installed_resp/piss-ant-pix2pix/' + trainoutputroot + '/tested_images',
 ]
processFoldersToMake = [
    '/content/installed_resp/piss-ant-pix2pix/p2p_process_tmp/', # dito
    '/content/installed_resp/piss-ant-pix2pix/p2p_process_tmp/_resized', # dito
    '/content/installed_resp/piss-ant-pix2pix/p2p_process_tmp/_blank', # dito
    '/content/installed_resp/piss-ant-pix2pix/p2p_process_tmp/_edges', # dito
    '/content/installed_resp/piss-ant-pix2pix/p2p_process_tmp/_combined', # dito
    '/content/installed_resp/piss-ant-pix2pix/p2p_process_tmp/_grayscale', # dito
    '/content/installed_resp/piss-ant-pix2pix/p2p_process_tmp/_native_edges',
    

]

In [ ]:
MakeDirs(ImgFoldersToMake)
MakeDirs(processFoldersToMake)

#inject all garbagepailkids jpg images recursive into png folder including rename to png

In [ ]:
#copy and rename params
dryRun = False
maxQty = 500
import shutil
orgFileExt = "jpg"
newFileExt = "png"

sourceFiles = recursive_glob(sourceFolder,'*.' + orgFileExt) # glob all jpg filepaths
iter = 0
#copy/rename loop
for fpath in sourceFiles[:maxQty]:
    if "_ch." not in fpath:
        fpathSplit = fpath.split('/')
        filename = fpathSplit[len(fpathSplit) - 1]
        newPath = os.path.join(destFolder ,filename.replace(orgFileExt, newFileExt))
        iter += 1
        print("[" +str(iter) + "/" + str(len(sourceFiles[:maxQty])) + "] copying " + fpath + " To " + newPath)
        if dryRun == False:
            shutil.copyfile(fpath, newPath )
            clear_output()
if dryRun == False:
    clear_output()

print("copied + renamed " + str(len(sourceFiles[:maxQty])) +" Files!")

## Preprocess Images in Folder to Pix2Pix sidebyside training set
# resize
# make edge or blank clone images
# combine org and cloned image 
# split into training and validation folders
# copy to final dataset folder

In [ ]:
!python /content/installed_resp/piss-ant-pix2pix/tools/process_custom.py --input_dir {destFolder} --operation resize --pad --size 256 --output_dir /content/installed_resp/piss-ant-pix2pix/p2p_process_tmp/_resized
clear_output()

In [ ]:
method ='blank' # edges grayscale native_edges or blank
if method =='edges': # makes clone in edges only 
    !python /content/installed_resp/piss-ant-pix2pix/tools/edge_custom.py --input_dir /content/installed_resp/piss-ant-pix2pix/p2p_process_tmp/_resized --output_dir  /content/installed_resp/piss-ant-pix2pix/p2p_process_tmp/_edges
if method =='blank': # makes clone with central rectangle cut out 
    !python /content/installed_resp/piss-ant-pix2pix/tools/process_custom.py --workers 1 --input_dir /content/installed_resp/piss-ant-pix2pix/p2p_process_tmp/_resized --operation blank --output_dir /content/installed_resp/piss-ant-pix2pix/p2p_process_tmp/_blank
if method =='grayscale': # makes grayscale clone 
    !python /content/installed_resp/piss-ant-pix2pix/tools/process_custom.py --workers 1 --input_dir /content/installed_resp/piss-ant-pix2pix/p2p_process_tmp/_resized --operation grayscale --output_dir /content/installed_resp/piss-ant-pix2pix/p2p_process_tmp/_grayscale

if method =='native_edges': # makes grayscale clone 
    !python /content/installed_resp/piss-ant-pix2pix/tools/process_custom.py --workers 1 --input_dir /content/installed_resp/piss-ant-pix2pix/p2p_process_tmp/_resized --operation edges --output_dir /content/installed_resp/piss-ant-pix2pix/p2p_process_tmp/_native_edges



#clear_output()


In [ ]:
#visual check if we have the same amount org images as clones
rezif = recursive_glob('/content/installed_resp/piss-ant-pix2pix/p2p_process_tmp/_resized','*.png')
combf = recursive_glob('/content/installed_resp/piss-ant-pix2pix/p2p_process_tmp/_' + method,'*.png')
print(str(len(rezif)) +"/"+ str(len(combf)))

In [ ]:

!python /content/installed_resp/piss-ant-pix2pix/tools/process_custom.py --input_dir /content/installed_resp/piss-ant-pix2pix/p2p_process_tmp/_resized --b_dir /content/installed_resp/piss-ant-pix2pix/p2p_process_tmp/_{method} --operation combine --output_dir /content/installed_resp/piss-ant-pix2pix/p2p_process_tmp/_combined
clear_output()

In [ ]:
# split in train and validate sets
!python /content/installed_resp/piss-ant-pix2pix/tools/split.py --dir /content/installed_resp/piss-ant-pix2pix/p2p_process_tmp/_combined
clear_output()

In [ ]:
# copy preprocessed files over to the final folder
!cp -rf /content/installed_resp/piss-ant-pix2pix/p2p_process_tmp/_combined/ {datasetFolder}
!cp -rf /content/installed_resp/piss-ant-pix2pix/p2p_process_tmp/_combined/ /content/drive/MyDrive/DataSets/GPK/
clear_output()

In [ ]:
# remove all process images
for f in processFoldersToMake:
    !rm -r {f}
# make clean folder for new process    
MakeDirs(processFoldersToMake)
#!rm -r /content/installed_resp/piss-ant-pix2pix/p2p_process_tmp/_combined/*.png


## 
> parser.add_argument("--input_dir", help="path to folder containing images")
> parser.add_argument("--mode", required=True, choices=["train", "test", "export"])
> parser.add_argument("--output_dir", required=True, help="where to put output files")
> parser.add_argument("--seed", type=int, default=0)
> parser.add_argument("--checkpoint", default=None, help="directory with checkpoint to resume training from or use for testing")

> parser.add_argument("--max_steps", type=int, help="number of training steps (0 to disable)")
> parser.add_argument("--max_epochs", type=int, default=3, help="number of training epochs")
> parser.add_argument("--summary_freq", type=int, default=100, help="update summaries every summary_freq steps")
> parser.add_argument("--progress_freq", type=int, default=50, help="display progress every progress_freq steps")
> parser.add_argument("--trace_freq", type=int, default=0, help="trace execution every trace_freq steps")
> parser.add_argument("--display_freq", type=int, default=0, help="write current training images every display_freq steps")
> parser.add_argument("--save_freq", type=int, default=100000, help="save model every save_freq steps, 0 to disable")
> parser.add_argument("--aspect_ratio", type=float, default=1.0, help="aspect ratio of output images (width/height)")
> parser.add_argument("--lab_colorization", action="store_true", help="split input image into brightness (A) and color (B)")
> parser.add_argument("--batch_size", type=int, default=1, help="number of images in batch")
> parser.add_argument("--which_direction", type=str, default="AtoB", choices=["AtoB", "BtoA"])
> parser.add_argument("--ngf", type=int, default=64, help="number of generator filters in first conv layer")
> parser.add_argument("--ndf", type=int, default=64, help="number of discriminator filters in first conv layer")
> parser.add_argument("--scale_size", type=int, default=266, help="scale images to this size before cropping to 256x256")
> parser.add_argument("--flip", dest="flip", action="store_true", help="flip images horizontally")
> parser.add_argument("--no_flip", dest="flip", action="store_false", help="don't flip images horizontally")

> parser.add_argument("--lr", type=float, default=0.00005, help="initial learning rate")
> parser.add_argument("--beta1", type=float, default=0.5, help="momentum term of adam")
> parser.add_argument("--l1_weight", type=float, default=0.0, help="weight on L1 term for generator gradient")
> parser.add_argument("--gan_weight", type=float, default=1.0, help="weight on GAN term for generator gradient")
> parser.add_argument("--skip_connection", type=int, default= 1, help="@luyi: wether to use skip connection")
> parser.add_argument("--patch_gan", type=int, default=1, help="@luyi: wether to use patch gan")
> parser.add_argument("--wgan", type=int, default=1, help="@luyi: wether to use wgan")


In [ ]:
# training i/o


training_epochs=3
print(train_metrics_output_folder)

##TODOS:
WARNING:tensorflow:From /content/installed_resp/piss-ant-pix2pix/pix2pix_custom.py:720: 
calling map_fn (from tensorflow.python.ops.map_fn) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Use fn_output_signature instead

WARNING:tensorflow:From /content/installed_resp/piss-ant-pix2pix/pix2pix_custom.py:760: 
Supervisor.__init__ (from tensorflow.python.training.supervisor) is deprecated and will be removed in a future version.
Instructions for updating:
Please switch to tf.train.MonitoredTrainingSession

WARNING:tensorflow:From /content/installed_resp/piss-ant-pix2pix/pix2pix_custom.py:272: 
string_input_producer (from tensorflow.python.training.input) is deprecated and will be removed in a future version.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.


WARNING:tensorflow:From /content/installed_resp/piss-ant-pix2pix/pix2pix_custom.py:273: 
WholeFileReader.__init__ (from tensorflow.python.ops.io_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.map(tf.read_file)`.

WARNING:tensorflow:From /content/installed_resp/piss-ant-pix2pix/pix2pix_custom.py:329: 
batch (from tensorflow.python.training.input) is deprecated and will be removed in a future version.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.batch(batch_size)` (or `padded_batch(...)` if `dynamic_pad=True`).
examples count = 3178


In [ ]:
# TRAIN!!!!!
retrain = ''
#retrain = '--checkpoint' + out_folder
!python /content/installed_resp/piss-ant-pix2pix/pix2pix_custom.py \
    --mode train \
    --no_flip \
    --lr 0.00004 \
    --display_freq 100 \
    --save_freq 100 \
    --batch_size 2 \
    --output_dir {train_metrics_output_folder} \
    --max_epochs {training_epochs} \
    --input_dir {train_input_folder} \
    --summary_freq 100 \
    {retrain} \
    --which_direction BtoA
#clear_output()

In [ ]:
# checkpoints
!ls -l {output_folder}

In [ ]:
# test the images in the validate folder

!python /content/installed_resp/piss-ant-pix2pix/pix2pix.py \
    --scale_size 1024 \
    --mode test \
    --output_dir {validate_output_images} \
    --input_dir {validate_input_folder} \
    --seed 4 \
    --checkpoint {train_metrics_output_folder}

clear_output()

In [ ]:
#!zip /content/installed_resp/piss-ant-pix2pix/img_align_celeba_png_resize/training_train/* /content/5000EpochsHotwheelsMetrics.zip

In [ ]:
!ls /content/

In [ ]:
# get an image
#!mkdir /content/installed_resp/images_to_test
#!mkdir /content/installed_resp/tested_images

#%cd /content/installed_resp/images_to_test

# get a few sketches to test

#!wget https://live.staticflickr.com/2936/33549790582_48a514baed_b.jpg

    
#%cd /content/installed_resp/piss-ant-pix2pix

#clear_output()

In [ ]:
# save the complete model
!python /content/installed_resp/piss-ant-pix2pix/server/tools/export-checkpoint.py --checkpoint /content/installed_resp/piss-ant-pix2pix/img_align_celeba_png_resize/_combined/train --output_file /content/installed_resp/piss-ant-pix2pix/img_align_celeba_png_resize/training_train/img_align_celeba_png_BtoA_model_save.bin
clear_output()

In [ ]:
#!cp /content/installed_resp/piss-ant-pix2pix/img_align_celeba_png_resize/training_train/checkpoint.zip /content/drive/MyDrive/train_metrics_hotwheels_png.zip

In [ ]:
#!zip /content/installed_resp/piss-ant-pix2pix/img_align_celeba_png_resize/training_train/* /content/installed_resp/piss-ant-pix2pix/train_metrics_hotwheels_png.zip
#!cp /content/installed_resp/piss-ant-pix2pix/train_metrics_hotwheels_png.zip ./img_align_celeba_png.zip

In [ ]:
# store zip to drive
#!zip /content/installed_resp/piss-ant-pix2pix/img_align_celeba_png/* /content/installed_resp/piss-ant-pix2pix/img_align_celeba_png_original_images.zip
#!zip /content/installed_resp/piss-ant-pix2pix/img_align_celeba_png_resize/_combined/* /content/installed_resp/piss-ant-pix2pix/img_align_celeba_png_dataset.zip
#!zip /content/installed_resp/piss-ant-pix2pix/img_align_celeba_png_resize/training_train/img_align_celeba_png_BtoA_model_save.bin /content/installed_resp/piss-ant-pix2pix/img_align_celeba_png_model_bin.zip
#!zip /content/installed_resp/piss-ant-pix2pix/img_align_celeba_png_resize/training_train/* /content/installed_resp/piss-ant-pix2pix/img_align_celeba_png.zip
# copy zips to drive
#%cd /content/drive/My\ Drive/Colab\ Notebooks/Best_Colabs
#!cp /content/installed_resp/piss-ant-pix2pix/img_align_celeba_png_original_images.zip img_align_celeba_png_original_images.zip
#!cp /content/installed_resp/piss-ant-pix2pix/img_align_celeba_png_dataset.zip ./img_align_celeba_png_dataset.zip
#!cp /content/installed_resp/piss-ant-pix2pix/img_align_celeba_png_model_bin.zip ./img_align_celeba_png_model_bin.zip
#!cp /content/installed_resp/piss-ant-pix2pix/img_align_celeba_png.zip ./img_align_celeba_png.zip

In [ ]:
import ipywidgets as widgets

slider = widgets.IntSlider(20, min=0, max=100)
slider

In [ ]:
import altair as alt
import ipywidgets as widgets
from vega_datasets import data

source = data.stocks()

stock_picker = widgets.SelectMultiple(
    options=source.symbol.unique(),
    value=list(source.symbol.unique()),
    description='Symbols')

# The value of symbols will come from the stock_picker.
@widgets.interact(symbols=stock_picker)
def render(symbols):
  selected = source[source.symbol.isin(list(symbols))]

  return alt.Chart(selected).mark_line().encode(
      x='date',
      y='price',
      color='symbol',
      strokeDash='symbol',
  )

In [ ]:
!mkdir /content/drive/MyDrive/HotWheelsImageSet_PNG

In [ ]:
!cp /content/installed_resp/piss-ant-pix2pix/img_align_celeba_png/*.png /content/drive/MyDrive/HotWheelsImageSet_PNG

## pretrained


In [ ]:
# TRAIN From checkpoint!!!!!
output_folder='/content/installed_resp/piss-ant-pix2pix/img_align_celeba_png_resize/training'
input_folder = '/content/installed_resp/piss-ant-pix2pix/img_align_celeba_png_resize/_combined/train'
training_epochs=20
!python /content/installed_resp/piss-ant-pix2pix/pix2pix_custom.py --mode train --display_freq 100 --save_freq 500 --batch_size 16 --output_dir {output_folder}_train --max_epochs {training_epochs} --input_dir {input_folder} --which_direction BtoA
#clear_output()